# Final Project
# “Artificial Intelligence - Wind Power Generation Forecasting”

**Load libraries**

In [ ]:
# Import all your libraries only in this cell
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime


In [ ]:
# import tensorflow libraries
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers

from tensorboard.plugins.hparams import api as hp
print(tf.__version__)

2.8.2


In [ ]:
#1 .- Download data from kaggle website
#2 .- Load data to drive.google.com
# connect goolge drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load datasets**

In [ ]:
## First way~~~ Kaggle API
## Connect to kaggle
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print("User uploade file '{name}' with length {length} bytes".format(name=fn,
        length = len(uploaded[fn])))
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json



Saving kaggle (2).json to kaggle (2) (4).json
User uploade file 'kaggle (2).json' with length 66 bytes
mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
#Check the files 
!kaggle competitions files aifpc2022

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
#download the dataset
!kaggle competitions download -c aifpc2022

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
#unzip files
!7za e aifpc2022.zip "train.csv"
!7za e aifpc2022.zip "test.csv"


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         
ERROR: No more files
aifpc2022.zip



System ERROR:
Unknown error -2147024872

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         
ERROR: No more files
aifpc2022.zip



System ERROR:
Unknown error -2147024872


In [ ]:
# load files
train_df = pd.read_csv('/content/test (2).csv')
test_df = pd.read_csv('/content/train (1).csv')

### second way

In [ ]:
#load dataset
train_df = pd.read_csv("/content/train (1).csv")
test_df = pd.read_csv("/content/test (2).csv")


In [ ]:
dataset=train_df.copy()
dataset.tail()

,ID,Date,Time,Wind_Energy_1,Wind_Energy_2,Wind_Energy_3
25148,25149,12/31/2020,20:00,0.0,0.3,1.6
25149,25150,12/31/2020,21:00,0.0,0.0,1.2
25150,25151,12/31/2020,22:00,0.0,0.4,2.1
25151,25152,12/31/2020,23:00,0.0,0.0,2.2
25152,25153,12/31/2020,24:00:00,0.0,0.0,2.6


In [ ]:
dataset_dp = dataset.drop((['ID','Date','Time']),axis=1)

In [ ]:
dataset_dp

,Wind_Energy_1,Wind_Energy_2,Wind_Energy_3
0,46.1,22.8,0.0
1,49.2,21.7,0.0
2,47.9,19.8,0.0
3,49.7,17.6,0.0
4,48.3,15.1,0.0
...,...,...,...
25148,0.0,0.3,1.6
25149,0.0,0.0,1.2
25150,0.0,0.4,2.1
25151,0.0,0.0,2.2


In [ ]:
dataset_dp.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Wind_Energy_1,25153.0,15.398925,18.931922,0.0,0.0,4.20,31.70,56.60
Wind_Energy_2,25153.0,8.536261,9.958963,0.0,0.0,3.20,17.70,54.90
Wind_Energy_3,25153.0,12.075927,11.549401,0.0,0.5,8.36,23.52,54.17


In [ ]:
dataset_dp.dtypes

Wind_Energy_1    float64
Wind_Energy_2    float64
Wind_Energy_3    float64
dtype: object

**Data cleaning**

In [ ]:
dataset.isna().sum()

Wind_Energy_1    0
Wind_Energy_2    0
Wind_Energy_3    0
Month            0
Days             0
Hour             0
dtype: int64

In [ ]:
dataset=dataset.dropna()

**preprocess** **data**



In [ ]:
dataset['Month'] = dataset.Date.str.split('/').str[0].astype(float)
dataset['Days'] = dataset.Date.str.split('/').str[1].astype(float)
dataset['Hour'] = dataset.Time.str.split(':').str[0].astype(float)

In [ ]:
test_df['Month'] = test_df.Date.str.split('/').str[0].astype(float)
test_df['Days'] = test_df.Date.str.split('/').str[1].astype(float)
test_df['Hour'] = test_df.Time.str.split(':').str[0].astype(float)

In [ ]:
test_df.drop(columns=['Date', 'Time', 'ID'], inplace=True )
test_df.tail()

,Month,Days,Hour
6979,10.0,18.0,20.0
6980,10.0,18.0,21.0
6981,10.0,18.0,22.0
6982,10.0,18.0,23.0
6983,10.0,18.0,24.0


In [ ]:
dataset.drop(columns=['Date', 'Time', 'ID'], inplace=True )
dataset.tail()

,Wind_Energy_1,Wind_Energy_2,Wind_Energy_3,Month,Days,Hour
25148,0.0,0.3,1.6,12.0,31.0,20.0
25149,0.0,0.0,1.2,12.0,31.0,21.0
25150,0.0,0.4,2.1,12.0,31.0,22.0
25151,0.0,0.0,2.2,12.0,31.0,23.0
25152,0.0,0.0,2.6,12.0,31.0,24.0


In [ ]:
dataset.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Wind_Energy_1,25153.0,15.398925,18.931922,0.0,0.0,4.20,31.70,56.60
Wind_Energy_2,25153.0,8.536261,9.958963,0.0,0.0,3.20,17.70,54.90
Wind_Energy_3,25153.0,12.075927,11.549401,0.0,0.5,8.36,23.52,54.17
Month,25153.0,6.758399,3.339400,1.0,4.0,7.00,10.00,12.00
Days,25153.0,15.834016,8.803905,1.0,8.0,16.00,23.00,31.00
Hour,25153.0,12.500457,6.922566,1.0,7.0,13.00,19.00,24.00


In [ ]:
## Split features
train_features = dataset.copy() 

train_labels = train_features[['Wind_Energy_1','Wind_Energy_2','Wind_Energy_3']].copy()
train_features = train_features.drop(['Wind_Energy_1','Wind_Energy_2','Wind_Energy_3'],
                                     axis = 1)

test_features_0 = test_df.copy()

In [ ]:
print(type(train_features))
i_train = train_features.values
print(type(i_train))
i_test =test_features_0.values
e_train= train_labels.values

print(i_train.shape)
print(i_test.shape)
print(e_train.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(25153, 3)
(6984, 3)
(25153, 3)


In [ ]:
print(type(train_features))
e_train = train_features.values
print(type(i_train))
e_test =test_features_0.values
e_train= train_labels.values

print(e_train.shape)
print(e_test.shape)
print(e_train.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(25153, 3)
(6984, 3)
(25153, 3)


In [ ]:
i_train_o = i_train.reshape(i_train.shape [0], 3 , 1)
i_test_o = i_test.reshape(i_test.shape [0], 3 , 1)


In [ ]:
e_train_o = e_train.reshape(i_train.shape [0], 3 , 1)
e_test_o = e_test.reshape(i_test.shape [0], 3 , 1)

**Normalization**

In [ ]:
#The first step is to create the norm layer
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
#fit the preprocessing layer
normalizer.adapt(np.array(i_train))


In [ ]:
print(normalizer.mean.numpy())

[[ 6.758 15.834 12.5  ]]


In [ ]:
first = np.array(i_train[:1])
with np.printoptions(precision =2, suppress = True):
  print('First example: ',first)
  print()
  print('Normalized', normalizer(first).numpy())

First example:  [[ 2. 17. 24.]]

Normalized [[-1.42  0.13  1.66]]


In [ ]:
from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
import keras.backend as K 
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential() # Sequeatial Model 
model.add(LSTM(128, input_shape=(3,1),activation = 'relu'))
model.add(Dense(30,activation = 'relu'))
model.add(Dense(3,activation = 'relu'))
model.compile(loss='mean_squared_error', optimizer = tf.optimizers.Adam(learning_rate = 0.0001),metrics =['accuracy']) 
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 128)               66560     
                                                                 
 dense_7 (Dense)             (None, 30)                3870      
                                                                 
 dense_8 (Dense)             (None, 3)                 93        
                                                                 
Total params: 70,523
Trainable params: 70,523
Non-trainable params: 0
_________________________________________________________________


In [ ]:
log_dir = 'logs/fit/LR_One_'+datetime.datetime.now().strftime('%Y%m%d-%H%m%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = log_dir, histogram_freq = 1
)
hparams_callback = hp.KerasCallback(log_dir,{
    'num_linear_units': 3,
})


In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

history = model.fit(i_train_o, e_train, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop], validation_split = 0.2)

##try predict four times

Epoch 1/100
671/671 [==============================] - 7s 9ms/step - loss: 303.5457 - accuracy: 0.2948 - val_loss: 254.9273 - val_accuracy: 0.3321
Epoch 2/100
671/671 [==============================] - 6s 9ms/step - loss: 285.1827 - accuracy: 0.3380 - val_loss: 252.8380 - val_accuracy: 0.3321
Epoch 3/100
671/671 [==============================] - 7s 10ms/step - loss: 282.3250 - accuracy: 0.3380 - val_loss: 251.5556 - val_accuracy: 0.3321
Epoch 4/100
671/671 [==============================] - 7s 10ms/step - loss: 280.5262 - accuracy: 0.3379 - val_loss: 251.7445 - val_accuracy: 0.3321
Epoch 5/100
671/671 [==============================] - 5s 7ms/step - loss: 278.8927 - accuracy: 0.3386 - val_loss: 250.8098 - val_accuracy: 0.3321
Epoch 6/100
671/671 [==============================] - 5s 7ms/step - loss: 277.2180 - accuracy: 0.3387 - val_loss: 249.9155 - val_accuracy: 0.3333
Epoch 7/100
671/671 [==============================] - 5s 7ms/step - loss: 275.5067 - accuracy: 0.3397 - val_loss: 2

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


,loss,accuracy,val_loss,val_accuracy,epoch
35,265.893982,0.337442,246.061218,0.334128,35
36,265.818512,0.338634,246.574905,0.332936,36
37,265.751709,0.338485,246.458328,0.334327,37
38,265.704437,0.338436,246.424698,0.332737,38
39,265.730408,0.338336,246.715210,0.333135,39


In [ ]:
def plot_loss(hist):
  plt.plot(hist.hist['loss'], label='loss')
  plt.plot(hist.hist['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Error [kWh]')
  plt.legend()
  plt.grid(True)


In [ ]:
test_df['Month'] = test_df.Date.str.split('/').str[0].astype(float)
test_df['Days'] = test_df.Date.str.split('/').str[1].astype(float)
test_df['Hour'] = test_df.Time.str.split(':').str[0].astype(float)
test_df = test_df.drop(columns=['Date','Time'])
test_df.tail()


,ID,Month,Days,Hour
6979,32133,10.0,18.0,20.0
6980,32134,10.0,18.0,21.0
6981,32135,10.0,18.0,22.0
6982,32136,10.0,18.0,23.0
6983,32137,10.0,18.0,24.0


In [ ]:
test_features = test_df.copy()
LR_One_df = model.predict(i_test_o)

In [ ]:
result = pd.DataFrame(LR_One_df, 
                      columns=['Wind_Energy_1','Wind_Energy_2','Wind_Energy_3'],
                      index = test_df['ID'])

In [ ]:
result.to_csv('/content/drive/MyDrive/Colab Notebooks/pipe.csv')